In [2]:
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.2 MB/s eta 0:00:00:00:01


In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [7]:
df = pd.read_csv("/kaggle/input/smaple-text/sample_text.csv")
df.shape

(8, 2)

In [8]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [9]:
from sentence_transformers import SentenceTransformer

In [10]:
encoder = SentenceTransformer('sentence-t5-large') 
vectors = encoder.encode(df.text)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
vectors.shape

(8, 768)

In [12]:
dim = vectors.shape[1]
dim

768

# Build a FAISS Index for vectors

In [13]:
import faiss
index= faiss.IndexFlatL2(dim)

In [14]:
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c916f4336f0> >

In [15]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

In [16]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

**Search for similar vector in the FAISS index created**

In [18]:
distances, I = index.search(svec, k=2)
distances

array([[0.59535456, 0.61831605]], dtype=float32)

In [19]:
I

array([[3, 2]])

In [20]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [21]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [22]:
search_query

'I want to buy a polo t-shirt'